<a href="https://colab.research.google.com/github/diputs03/AI-Studies/blob/main/From-tensorflow-mnist-tutorial/dynamic_architect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#@title Aiming a Dynaimic Graph-structured NeuronNetwork
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sympy

In [34]:
#@title Model Class
class Model:
  def relu(x) -> float:
    if x >= 0: return x
    else: return 0
  def sigmoid(x) -> float:
    return 1/(1 + np.e**(-x))
  def linear(x) -> float:
    return x
  def nul(x) -> float:
    return x
# Neuron
  class Neuron:
    name = ''
    prev, next = [], []
    bias = 0
    func = None
    def __init__(self, name, prev, next, func) -> None:
      self.name = name
      self.prev = prev
      self.next = next
      self.func = func
    def __str__(self) -> str:
      ret = str(self.name) + '('
      for n in self.next:
        ret += str(n)
      return ret + ')'
# Model handles
  Input_layer, Output_layer, all_neurons = [], [], []
  links = {}
  def __init__(self, input_size, output_size):
    for i in range(input_size):
      self.Input_layer += [Model.Neuron(name=f'input_{i}',
                                        prev=[], next=[], func=Model.sigmoid)]
    for o in range(output_size):
      self.Output_layer += [Model.Neuron(name=f'output_{o}',
                                         prev=[], next=[], func=Model.sigmoid)]
    for u in self.Input_layer:
      for v in self.Output_layer:
        self.links[(u,v)] = 0
        u.next.append(v)
        v.prev.append(u)
    self.all_neurons += self.Input_layer+self.Output_layer

  def __len__(self):
    return len(self.all_neurons)

  def forward(self, X):
    for x in X: assert len(x) == len(self.Input_layer), 'invalid input'
    l = len(X)
    stack = self.Input_layer.copy()
# we have to cache the input in something because this is not a layered graph
    cache, cnt = {}, {}
    for n in self.all_neurons:
      cache[n], cnt[n] = [0] * l, 0
    for e in range(l):
      for i, elem in enumerate(self.Input_layer):
        cache[elem][e] = X[e][i]
# stack for nodes to be pushed
    while len(stack) != 0:
      c = stack.pop()
      for e in range(l):
        cache[c][e] = c.func(cache[c][e]+c.bias)
      for n in c.next:
        for e in range(l):
          cache[n][e] += self.links[(c,n)] * cache[c][e]
        cnt[n] += 1
        if cnt[n] == len(n.prev):
          stack.append(n)
    return cache

  def eval(self, X):
    cache = self.forward(X)
    l = len(X)
    ret = [[] for _ in range(l)]
    for e in range(l):
      for o in self.Output_layer:
        ret[e] += [cache[o][e]]
    return ret

  def backward(self, X, Y, learning_rate):
    assert len(X) == len(Y), 'invalid input'
    for x in X: assert len(x) == len(self.Input_layer), 'invalid input'
    for y in Y: assert len(y) == len(self.Output_layer), 'invalid input'
    l = len(X)
    cache = self.forward(X)
    stack = self.Output_layer.copy()
    delta, cnt = {}, {}
    for n in self.all_neurons:
      delta[n], cnt[n] = [0] * l, 0
    for e in range(l):
      for o, elem in enumerate(self.Output_layer):
        delta[elem][e] = Y[e][o] - cache[elem][e]

    while len(stack) != 0:
      c = stack.pop()
      x = sympy.symbols('x')
      deri = sympy.diff(c.func(x), x)
      for e in range(l):
        delta[c][e] *= deri.evalf(subs={x: cache[c][e]})
      for p in c.prev:
        for e in range(l):
          delta[p][e] += self.links[(p,c)] * delta[c][e]
        cnt[p] += 1
        self.links[(p,c)] += np.dot(delta[c], cache[c]) * learning_rate
        c.bias += np.sum(delta[c], axis=0) * learning_rate
        if cnt[p] == len(p.next):
          stack.append(p)
    return None

  def train(self, X, Y, epochs, learning_rate):
    l = len(X)
    for epoch in range(epochs):
        output = self.eval(X)
        self.backward(X, Y, learning_rate)
        if epoch % 40 == 0:
            loss = 0
            for e in range(l):
              for o in range(len(self.Output_layer)):
                loss += (Y[e][o]-output[e][o]) ** 2
            loss = (loss) ** 0.5
            print(f"Epoch {epoch}, Loss:{loss}")

  def __str__(self):
    ret=''
    for i in self.Input_layer:
      ret += i.__str__()
    return ret

# tests
  def Add_Node(self):
    assert(len(self.Input_layer) != 0)
    assert(len(self.Output_layer) != 0)
    u = self.Input_layer[0]
    v = self.Output_layer[0]
    self.links.pop((u, v))
    n = Model.Neuron(name=f'mid_{0}', prev=[u], next=[v], func=Model.sigmoid)
    self.links[(u, n)] = 1
    self.links[(n, v)] = 1
    self.all_neurons.append(n)
    u.next.remove(v)
    u.next.append(n)
    v.prev.remove(u)
    v.prev.append(n)
    print('new node', n)

# testing code
mod=Model(2, 2)
print(mod.Input_layer)
print(mod.Output_layer)
print(mod.eval([[1, 2], [1, 0]]))
mod.train([[1, 2]], [[1, 0]], 800, 10)
mod.Add_Node()
print(mod.eval([[1, 2], [1, 0]]))
mod.train([[1, 2]], [[1, 0]], 800, 10)
#mod.plot()
print(mod)

[<__main__.Model.Neuron object at 0x7a9d83f34f50>, <__main__.Model.Neuron object at 0x7a9d96e2c350>]
[<__main__.Model.Neuron object at 0x7a9d83f37850>, <__main__.Model.Neuron object at 0x7a9d83f36950>]
[[0.5, 0.5], [0.5, 0.5]]
Epoch 0, Loss:0.7071067811865476
Epoch 40, Loss:0.00543523048398142
Epoch 80, Loss:0.00286793656265812
Epoch 120, Loss:0.00194967013188920
Epoch 160, Loss:0.00147724966823401
Epoch 200, Loss:0.00118927227142813
Epoch 240, Loss:0.000995324680317659
Epoch 280, Loss:0.000855801043747491
Epoch 320, Loss:0.000750604786910675
Epoch 360, Loss:0.000668451426962564
Epoch 400, Loss:0.000602515060154402
Epoch 440, Loss:0.000548423997446359
Epoch 480, Loss:0.000503248590933190
Epoch 520, Loss:0.000464951870085756
Epoch 560, Loss:0.000432073565181824
Epoch 600, Loss:0.000403539483024824
Epoch 640, Loss:0.000378541811870924
Epoch 680, Loss:0.000356461360821204
Epoch 720, Loss:0.000336815538100205
Epoch 760, Loss:0.000319222647717711
new node mid_0(output_0())
[[0.9995973758776

In [35]:
mod.eval([[1, 2]])

[[0.999879025057640, 0.000124435789109206]]

In [ ]:
#@title plotting the network, TODO://
  def plot(self):
    def blank_diagram(fig_width=16, fig_height=9,
                  bg_color="antiquewhite", color="midnightblue"):
      fig = plt.figure(figsize=(fig_width / 2.54, fig_height / 2.54))
      ax = fig.add_axes((0, 0, 1, 1))
      ax.set_xlim(0, fig_width)
      ax.set_ylim(0, fig_height)
      ax.set_facecolor(bg_color)

      ax.tick_params(bottom=False, top=False,
                    left=False, right=False)
      ax.tick_params(labelbottom=False, labeltop=False,
                    labelleft=False, labelright=False)

      ax.spines["top"].set_color(color)
      ax.spines["bottom"].set_color(color)
      ax.spines["left"].set_color(color)
      ax.spines["right"].set_color(color)
      ax.spines["top"].set_linewidth(4)
      ax.spines["bottom"].set_linewidth(4)
      ax.spines["left"].set_linewidth(4)
      ax.spines["right"].set_linewidth(4)

      return fig, ax
    fig, ax = blank_diagram()

    centers = [(3.5, 6.5), (8, 6.5), (12.5, 6.5), (8, 2.5)]
    radii = 1.5
    texts = [
        "\n".join(["My roommate", "is a Philistine", "and a boor"]),
        "\n".join(["My roommate", "ate the last", "of the", "cold cereal"]),
        "\n".join(["I am really", "really hungy"]),
        "\n".join(["I'm annoyed", "at my roommate"]),
    ]

    # Draw circles with text in the center

    for i, center in enumerate(centers):
        x, y = center
        theta = np.linspace(0, 2 * np.pi, 100)
        ax.plot(
            x + radii * np.cos(theta),
            y + radii * np.sin(theta),
            color="midnightblue",
        )
        ax.text(
            x, y,
            texts[i],
            horizontalalignment="center",
            verticalalignment="center",
            color="midnightblue",
        )

    ax.annotate(
        "",
        (centers[1][0] - radii, centers[1][1]),
        (centers[0][0] + radii, centers[0][1]),
        arrowprops=dict(arrowstyle = "-|>"),
    )
    ax.annotate(
        "",
        (centers[2][0] - radii, centers[2][1]),
        (centers[1][0] + radii, centers[1][1]),
        arrowprops=dict(arrowstyle = "-|>"),
    )
    ax.annotate(
        "",
        (centers[3][0] - .7 * radii, centers[3][1] + .7 * radii),
        (centers[0][0] + .7 * radii, centers[0][1] - .7 * radii),
        arrowprops=dict(arrowstyle = "-|>"),
    )
    ax.annotate(
        "",
        (centers[3][0] + .7 * radii, centers[3][1] + .7 * radii),
        (centers[2][0] - .7 * radii, centers[2][1] - .7 * radii),
        arrowprops=dict(arrowstyle = "-|>"),
    )
    fig.show()